In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
#from catboost import CatBoostRegressor
#!pip install catboost

In [9]:
#ESTO NO LO EJECUTEN SI UTILIZAN UNA VERSION DE PC , YO LO HAGO PARA PODER USAR EL TRAIN DE MI DRIVE,Y USAR EL GITHUB ONLINE
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)




In [10]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1PFclWqu_bzIwLYdV12S3d8pS5XenxPeG'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train.csv')

In [11]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
test_id = '1uPd0roB5CyXnjXHm3R_0QqKcoaCZyrpX'
downloaded = drive.CreateFile({'id': test_id})
downloaded.GetContentFile('test.csv')

In [12]:
data = pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
data2=pd.read_csv('train.csv')

In [6]:

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
test_id = '18Ab39T1hPgTBv58oXdVk0NfBE36Fp21D'
downloaded = drive.CreateFile({'id': test_id})
downloaded.GetContentFile('test_3.csv')

In [7]:
#esta linea se cambia para cuando formamos el test, sino lo comentamos para
#que forme el train
#data=test

# funciones a usar

In [ ]:
#unificamos locationes

In [ ]:
#hallamos paises and ciudades en los tweet
def find_location(tweet):
    
    for location in locations_list:
        if str(location) in tweet:
            return str(location)
    return 'not_location'

In [13]:
import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

  #FUNCIONES UTILES

def only_letters(tweet):
      tweet = re.sub(r'http\S*', '', tweet)
      tweet = re.sub(r'[^a-z\s]', '', tweet)
      return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
      filtered_words=[]
      for w in tokenized_text:
          if w not in stop_words:
              filtered_words.append(w)
      return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
      lemmatized_words = []
      for word in tweet:
          lemmatized_words.append(lemmatizer.lemmatize(word))
      return lemmatized_words

def transform_to_text(tweet_words):
      return " ".join(tweet_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [14]:
#limpiando el texto

def clean_text(train):

  import nltk
  nltk.download('stopwords') 
  nltk.download('punkt')
  nltk.download('wordnet')
  from nltk.tokenize import word_tokenize
  from nltk.corpus import stopwords
  from nltk.stem.wordnet import WordNetLemmatizer

  #FUNCIONES UTILES

  def only_letters(tweet):
      tweet = re.sub(r'http\S*', '', tweet)
      tweet = re.sub(r'[^a-z\s]', '', tweet)
      return tweet

  stop_words=set(stopwords.words("english"))

  def filter_stopwords(tokenized_text):
      filtered_words=[]
      for w in tokenized_text:
          if w not in stop_words:
              filtered_words.append(w)
      return filtered_words

  lemmatizer = WordNetLemmatizer()

  def lemmatize_tweet(tweet):
      lemmatized_words = []
      for word in tweet:
          lemmatized_words.append(lemmatizer.lemmatize(word))
      return lemmatized_words

  def transform_to_text(tweet_words):
      return " ".join(tweet_words)

  #Cleaning text

  train['clean_text'] = train['text'].str.lower()

  train['clean_text'] = train['clean_text'].apply(only_letters)    

  #Tokenización

  train['clean_text'] = train['clean_text'].apply(word_tokenize)

  #Remove stopwords

  train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

  #Lemmatization                                                                       

  train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

  train['clean_text'] = train['clean_text'].apply(transform_to_text)

In [ ]:
#descubriendo nuevos features

In [15]:
data.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [16]:
data.groupby('location').agg(cantidad=('location','count')).nlargest(30,'cantidad')[:50]

,cantidad
location,
USA,104
New York,71
United States,50
London,45
Canada,29
Nigeria,28
UK,27
"Los Angeles, CA",26
India,24


In [ ]:
#ahora vimos arriba que por lo menos las primeras 50 son relevantes osea mayor a 10 tweet, 
#seguramente dentro de location debe haber las mismas ciudades/paises escritos de otra forma
#unificamos con esos 50 de arriba


In [17]:
def unificar_cuidades_paies(data2):
      data2.loc[data2['location'].str.contains('(?i)flori')==True,'location' ]='Florida'
      data2.loc[data2['location'].str.contains('(?i)florida')==True,'location' ]='Florida'
      data2.loc[data2['location'].str.contains('(?i), fl')==True,'location' ]='Florida'
      data2.loc[data2['location'].str.contains('(?i)toronto')==True,'location' ]='toronto'
      data2.loc[data2['location'].str.contains('(?i)carolina')==True,'location' ]='carolina'
      data2.loc[data2['location'].str.contains('(?i)colorado')==True,'location' ]='colorado'
      data2.loc[data2['location'].str.contains('(?i)ny')==True,'location' ]='New York, NY'
      data2.loc[data2['location'].str.contains('(?i)new yo')==True,'location' ]='New York, NY'
      data2.loc[data2['location'].str.contains('(?i)california')==True,'location' ]='California'
      data2.loc[data2['location'].str.contains('(?i), pa')==True,'location' ]='Philadelphia'
      data2.loc[data2['location'].str.contains('(?i),phila')==True,'location' ]='Philadelphia'
      data2.loc[data2['location'].str.contains('(?i),phili')==True,'location' ]='Philippines'  
      data2.loc[data2['location'].str.contains('(?i)Massa')==True,'location' ]='Massachusetts'
      data2.loc[data2['location'].str.contains('(?i)Michi')==True,'location' ]='Michigan'
      data2.loc[data2['location'].str.contains('(?i)Ariz')==True,'location' ]='Arizona'  
      data2.loc[data2['location'].str.contains('(?i), nc')==True,'location' ]='New Jersey'
      data2.loc[data2['location'].str.contains('(?i)jerse')==True,'location' ]='New Jersey'  
      data2.loc[data2['location'].str.contains('(?i)canad')==True,'location' ]='Canada'
      data2.loc[data2['location'].str.contains('(?i)niger')==True,'location' ]='Nigeria'
      data2.loc[data2['location'].str.contains('(?i)indi')==True,'location' ]='India'
      data2.loc[data2['location'].str.contains('(?i)flori')==True,'location' ]='Florida'
      data2.loc[data2['location'].str.contains('(?i)london')==True,'location' ]='Florida'
      data2.loc[data2['location'].str.contains('USA')==True,'location' ]='USA'   
      data2.loc[data2['location'].str.contains('United States')==True,'location' ]='USA' 
      

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [19]:
#ahi seleccionamos, ahora aplicamos
unificar_cuidades_paies(data)

In [20]:
#vemos que aumento su cantidad respecto al anterior de las localitaciones
data.groupby('location').agg(cantidad=('location','count')).nlargest(30,'cantidad')[:5]

,cantidad
location,
"New York, NY",295
USA,265
Florida,204
Canada,84
California,82


In [ ]:
#ahora la idea es agarrar las localitaciones que no utilizamos en 
# nuestras primer analisis y splitearlo en n-gramas
#y luego ordenarlo de mayor a menor frecuencia

In [21]:
#datashit=data[data['location'].notnull()]
sacar_columnas_usuales=data.groupby('location').agg(cantidad=('location','count')).nlargest(30,'cantidad')[:50]

In [22]:
sacar_columnas_usuales=list(sacar_columnas_usuales.index)

In [23]:
sacar_columnas_usuales[:3]

['New York, NY', 'USA', 'Florida']

In [24]:
#filtramos las locaciones que ya utilizamos
data_find_new_location=data[data['location'].notnull()]

data_find_new_location=data_find_new_location[data_find_new_location['location'].isin(sacar_columnas_usuales)==False]

In [25]:
data_find_new_location.head(3)

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1


In [ ]:
#guardo todas esas locaciones en una lista para filtrarla y sacarle stop word

In [26]:
locacionesx=list(data_find_new_location['location'])
texto=''
for loca in locacionesx:
  texto= texto + ' ' + loca
texto

" Birmingham Est. September 2012 - Bristol AFRICA Pretoria World Wide!! Paranaque City Live On Webcam milky way Live On Webcam England. Sheffield Township, Ohio Barbados Anaheim Abuja South Africa Sao Paulo, Brazil hollywoodland  Edmonton, Alberta - Treaty 6 Inang Pamantasan Twitter Lockout in progress Concord, CA Calgary, AB Birmingham CLVLND Nashville, TN Santa Clara, CA St. Louis, MO Walker County, Alabama San Mateo County, CA Your Sister's Bedroom Arlington, TX South Bloomfield, OH Maldives Manchester, NH global Alberta | Sask. | Montana Charlotte Baton Rouge, LA Hagerstown, MD Gloucestershire , UK Instagram - @heyimginog  304 Switzerland 304 US 304 Instagram - @heyimginog  304 Somewhere Only We Know ? Belgium Switzerland US dope show Switzerland Switzerland Baker City Oregon 304 304 marysville ca  304 Hermosa Beach, CA 304 304 19.600858, -99.047821 Pennsylvania Salt Lake City, Utah Palo Alto, CA Eagle Pass, Texas bangalore Financial News and Views y(our) boyfriends legs  Somewhere

In [ ]:
#listo se limpio el texto

In [27]:
texto2=texto
texto2=only_letters(texto2) 
texto2=word_tokenize(texto2)
texto2=filter_stopwords(texto2)
texto2=lemmatize_tweet(texto2)
texto2=transform_to_text(texto2)
texto2

'irmingham st eptember ristol retoria orld ide aranaque ity ive n ebcam milky way ive n ebcam ngland heffield ownship hio arbados naheim buja outh frica ao aulo razil hollywoodland dmonton lberta reaty nang amantasan witter ockout progress oncord algary irmingham ashville anta lara ouis alker ounty labama ateo ounty isters edroom rlington outh loomfield aldives anchester global lberta ask ontana harlotte aton ouge agerstown loucestershire nstagram heyimginog witzerland nstagram heyimginog omewhere nly e elgium witzerland dope show witzerland witzerland aker ity regon marysville ca ermosa ennsylvania alt ake ity tah alo lto agle as exas bangalore inancial ew iews boyfriend leg omewhere ut sri lanka ot resident hrissur avenford srael ashion eaven rancisco italy ackson ew rleans est ale appily arried kid ambridge rizona msterdam windonngland appily arried kid illiamstown arachi appily arried kid appily arried kid c h c g exington laya igher lace orsemind oksburg reater anchester oston ano

In [ ]:
#ahora hay que spliteralo en n-gramas, y ordenarlo ascendentemente por frecuencia

In [28]:
import collections
from nltk import ngrams
import itertools
acum=list()
sentence = texto2
n = 6
sixgrams = ngrams(sentence.split(), n)

for grams in sixgrams:
     acum.append(list(grams))

In [29]:
acum[:5]

[['irmingham', 'st', 'eptember', 'ristol', 'retoria', 'orld'],
 ['st', 'eptember', 'ristol', 'retoria', 'orld', 'ide'],
 ['eptember', 'ristol', 'retoria', 'orld', 'ide', 'aranaque'],
 ['ristol', 'retoria', 'orld', 'ide', 'aranaque', 'ity'],
 ['retoria', 'orld', 'ide', 'aranaque', 'ity', 'ive']]

In [30]:
#como es una lista de lista,entonces unifico todo en una sola lista
merged = list(itertools.chain(*acum))  
merged[:5]

['irmingham', 'st', 'eptember', 'ristol', 'retoria']

In [ ]:
#ordeno ascendentemente por frecuencia

In [31]:
dfd=pd.DataFrame(merged)
dfd=dfd.groupby(0).agg(cant=(0,'count')).nlargest(100,'cant').reset_index()
dfd.columns=['loca','cant']
dfd[:5]


,loca,cant
0,ity,354
1,ngland,330
2,ew,294
3,exas,268
4,outh,264


In [32]:
#creo otro unificador de localizaciones con estas palabras
def segundaunificacion(data2):
      data2.loc[data2['location'].str.contains('(?i)ngland')==True,'location' ]='England'
      data2.loc[data2['location'].str.contains('(?i)exas')==True,'location' ]='Texas'
      data2.loc[data2['location'].str.contains('(?i)ustralia')==True,'location' ]='Australia'
      data2.loc[data2['location'].str.contains('(?i)ashington')==True,'location' ]='Washington'
      data2.loc[data2['location'].str.contains('(?i)tlanta')==True,'location' ]='Atlanta'
      data2.loc[data2['location'].str.contains('(?i)eattle')==True,'location' ]='beattle'
      data2.loc[data2['location'].str.contains('(?i)anchester')==True,'location' ]='Manchester'
      data2.loc[data2['location'].str.contains('(?i)frica')==True,'location' ]='Africa'
      data2.loc[data2['location'].str.contains('(?i)regon')==True,'location' ]='Oregon'
      data2.loc[data2['location'].str.contains('(?i)ydney')==True,'location' ]='sydney'
      data2.loc[data2['location'].str.contains('(?i)ouston')==True,'location' ]='Houston'
      data2.loc[data2['location'].str.contains('(?i)iego')==True,'location' ]='Diego'
      data2.loc[data2['location'].str.contains('(?i)cotland')==True,'location' ]='Scotland'
      data2.loc[data2['location'].str.contains('(?i)rancisco')==True,'location' ]='Francisco'
      data2.loc[data2['location'].str.contains('(?i)eorgia')==True,'location' ]='Georgia'
      data2.loc[data2['location'].str.contains('(?i)allas')==True,'location' ]='Dallas'
      data2.loc[data2['location'].str.contains('(?i)ancouver')==True,'location' ]='Dancouver'
      data2.loc[data2['location'].str.contains('(?i)elbourne')==True,'location' ]='Melbourne'
      data2.loc[data2['location'].str.contains('(?i)ingdom')==True,'location' ]='Kingdom'
      data2.loc[data2['location'].str.contains('(?i)algary')==True,'location' ]='Calgary'
      data2.loc[data2['location'].str.contains('(?i)apan')==True,'location' ]='Japan'
      data2.loc[data2['location'].str.contains('(?i)hilippines')==True,'location' ]='Philippines'
      data2.loc[data2['location'].str.contains('(?i)klahoma')==True,'location' ]='Oklahoma'
      data2.loc[data2['location'].str.contains('(?i)ortland')==True,'location' ]='Portland'
      data2.loc[data2['location'].str.contains('(?i)acramento')==True,'location' ]='Sacramento'
      data2.loc[data2['location'].str.contains('(?i)oston')==True,'location' ]='Hoston'
      data2.loc[data2['location'].str.contains('(?i)ustin')==True,'location' ]='justin'
      data2.loc[data2['location'].str.contains('(?i)ashville')==True,'location' ]='ashville'
      data2.loc[data2['location'].str.contains('(?i)hicago')==True,'location' ]='chicago'
      data2.loc[data2['location'].str.contains('(?i)merica')==True,'location' ]='America'
      data2.loc[data2['location'].str.contains('(?i)Vegas')==True,'location' ]='Vegas'
      data2.loc[data2['location'].str.contains('(?i)emphis')==True,'location' ]='Memphis'
      data2.loc[data2['location'].str.contains('(?i)Ohio')==True,'location' ]='Ohio'
      data2.loc[data2['location'].str.contains('(?i)ictoria')==True,'location' ]='Victoria'
      data2.loc[data2['location'].str.contains('(?i)ONTARIO')==True,'location' ]='Ontario'
      data2.loc[data2['location'].str.contains('(?i)reland')==True,'location' ]='Ireland'
      data2.loc[data2['location'].str.contains('(?i)irmingham')==True,'location' ]='Birmingham'
      data2.loc[data2['location'].str.contains('(?i)ennessee')==True,'location' ]='Tennessee'
      data2.loc[data2['location'].str.contains('(?i)ewcastle')==True,'location' ]='Newcastle'
      data2.loc[data2['location'].str.contains('(?i)olumbia')==True,'location' ]='Columbia'
      data2.loc[data2['location'].str.contains('(?i)Spain')==True,'location' ]='Spain'
      data2.loc[data2['location'].str.contains('(?i)ansas')==True,'location' ]='Kansas'
      data2.loc[data2['location'].str.contains('(?i)Denver')==True,'location' ]='Colorado'
      
      data2.loc[data2['location'].str.contains('(?i)Louis')==True,'location' ]='Louis'

      data2.loc[data2['location'].str.contains('(?i)rance')==True,'location' ]='France'
      data2.loc[data2['location'].str.contains('(?i)etherlands')==True,'location' ]='Netherlands'
      data2.loc[data2['location'].str.contains('(?i)weden')==True,'location' ]='Sweden'
      data2.loc[data2['location'].str.contains('(?i)akistan')==True,'location' ]='Pakistan'
      data2.loc[data2['location'].str.contains('(?i)ritish')==True,'location' ]='British'
      data2.loc[data2['location'].str.contains('(?i)brasil')==True,'location' ]='Brasil'
      data2.loc[data2['location'].str.contains('(?i)rome')==True,'location' ]='Rome'
      data2.loc[data2['location'].str.contains('(?i)leveland')==True,'location' ]='cleveland'
      data2.loc[data2['location'].str.contains('(?i)valley')==True,'location' ]='alley'
      data2.loc[data2['location'].str.contains('(?i)ingapore')==True,'location' ]='singapore'
      data2.loc[data2['location'].str.contains('(?i)arizona')==True,'location' ]='arizona'
      data2.loc[data2['location'].str.contains('(?i)llinois')==True,'location' ]='illinois'
      data2.loc[data2['location'].str.contains('(?i)tokyo')==True,'location' ]='tokyo'

In [33]:
#ahora unifico de nuevo el dataset, pero veremos el antes y despues
data.groupby('location').agg(cantidad=('location','count')).nlargest(30,'cantidad')[:20]


,cantidad
location,
"New York, NY",295
USA,265
Florida,204
Canada,84
California,82
New Jersey,69
India,68
Nigeria,61
Philadelphia,33


In [34]:
segundaunificacion(data)

In [35]:
#despues
data.groupby('location').agg(cantidad=('location','count')).nlargest(50,'cantidad')

,cantidad
location,
"New York, NY",295
USA,265
Florida,204
Canada,84
California,82
New Jersey,69
India,68
Washington,68
Australia,65


In [ ]:
#ya unificamos por palabras mas usuales en el train, ahora vamos aplicar esas funciones al test
#y guardarlo en un csv.

In [39]:
#antes
test.groupby('location').agg(cantidad=('location','count')).nlargest(10,'cantidad')

,cantidad
location,
New York,38
USA,37
Worldwide,16
United States,15
Canada,13
London,13
Kenya,12
Nigeria,12
United Kingdom,12


In [40]:

unificar_cuidades_paies(test)
segundaunificacion(test)


In [41]:
#despues
test.groupby('location').agg(cantidad=('location','count')).nlargest(10,'cantidad')

,cantidad
location,
"New York, NY",128
USA,108
Florida,65
Canada,41
England,32
India,32
California,29
Washington,29
New Jersey,28


In [43]:
#tambien vamos a aplicar clean_text asi ya esta formateado para uso
clean_text(test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [55]:
clean_text(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [56]:
data.head(1)

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u


In [ ]:
#ahora si guardamos el train y test con columnas unificadas

In [57]:
from google.colab import files
test.to_csv('test_uni_loca_1.csv') 
files.download('test_uni_loca_1.csv')
data.to_csv('data_uni_loca_1.csv') 
files.download('data_uni_loca_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ahora completamos deacuerdo a la localizacion que se encuentre en el tweet( solo localitaziones nulas)

In [ ]:
#completar las localizaciones fatantes con alguna palabra de pais( to 50 mas comunes, con cant tweet relevantes MAYOR A 13) contenido en el texto(tweet)

In [58]:
#localions_list_list = list(set(data.keyword))  #todas las localizaciones

data_con_localizaciones_usuales=data.groupby('location').agg(cantidad=('location','count')).nlargest(50,'cantidad')
locations_list = list(data_con_localizaciones_usuales.index)
locations_list[:5]

['New York, NY', 'USA', 'Florida', 'Canada', 'California']

In [59]:
#hallamos paises and ciudades en los tweet
def unificar_localization_with_tweet(data2):
      data2.loc[data2['clean_text'].str.contains('(?i)flori')==True,'location' ]='Florida'
      data2.loc[data2['clean_text'].str.contains('(?i)florida')==True,'location' ]='Florida'
      data2.loc[data2['clean_text'].str.contains('(?i), fl')==True,'location' ]='Florida'
      data2.loc[data2['clean_text'].str.contains('(?i)toronto')==True,'location' ]='toronto'
      data2.loc[data2['clean_text'].str.contains('(?i)carolina')==True,'location' ]='carolina'
      data2.loc[data2['clean_text'].str.contains('(?i)colorado')==True,'location' ]='colorado'
      data2.loc[data2['clean_text'].str.contains('(?i)ny')==True,'location' ]='New York, NY'
      data2.loc[data2['clean_text'].str.contains('(?i)new yo')==True,'location' ]='New York, NY'
      data2.loc[data2['clean_text'].str.contains('(?i)california')==True,'location' ]='California'
      data2.loc[data2['clean_text'].str.contains('(?i), pa')==True,'location' ]='Philadelphia'
      data2.loc[data2['clean_text'].str.contains('(?i),phila')==True,'location' ]='Philadelphia'
      data2.loc[data2['clean_text'].str.contains('(?i),phili')==True,'location' ]='Philippines'  
      data2.loc[data2['clean_text'].str.contains('(?i)Massa')==True,'location' ]='Massachusetts'
      data2.loc[data2['clean_text'].str.contains('(?i)Michi')==True,'location' ]='Michigan'
      data2.loc[data2['clean_text'].str.contains('(?i)Ariz')==True,'location' ]='Arizona'  
      data2.loc[data2['clean_text'].str.contains('(?i), nc')==True,'location' ]='New Jersey'
      data2.loc[data2['clean_text'].str.contains('(?i)jerse')==True,'location' ]='New Jersey'  
      data2.loc[data2['clean_text'].str.contains('(?i)canad')==True,'location' ]='Canada'
      data2.loc[data2['clean_text'].str.contains('(?i)niger')==True,'location' ]='Nigeria'
      data2.loc[data2['clean_text'].str.contains('(?i)indi')==True,'location' ]='India'
      data2.loc[data2['clean_text'].str.contains('(?i)flori')==True,'location' ]='Florida'
      data2.loc[data2['clean_text'].str.contains('(?i)london')==True,'location' ]='Florida'
      data2.loc[data2['clean_text'].str.contains('USA')==True,'location' ]='USA'   
      data2.loc[data2['clean_text'].str.contains('United States')==True,'location' ]='USA' 

      data2.loc[data2['clean_text'].str.contains('(?i)ngland')==True,'location' ]='England'
      data2.loc[data2['clean_text'].str.contains('(?i)exas')==True,'location' ]='Texas'
      data2.loc[data2['clean_text'].str.contains('(?i)ustralia')==True,'location' ]='Australia'
      data2.loc[data2['clean_text'].str.contains('(?i)ashington')==True,'location' ]='Washington'
      data2.loc[data2['clean_text'].str.contains('(?i)tlanta')==True,'location' ]='Atlanta'
      data2.loc[data2['clean_text'].str.contains('(?i)eattle')==True,'location' ]='beattle'
      data2.loc[data2['clean_text'].str.contains('(?i)anchester')==True,'location' ]='Manchester'
      data2.loc[data2['clean_text'].str.contains('(?i)frica')==True,'location' ]='Africa'
      data2.loc[data2['clean_text'].str.contains('(?i)regon')==True,'location' ]='Oregon'
      data2.loc[data2['clean_text'].str.contains('(?i)ydney')==True,'location' ]='sydney'
      data2.loc[data2['clean_text'].str.contains('(?i)ouston')==True,'location' ]='Houston'
      data2.loc[data2['clean_text'].str.contains('(?i)iego')==True,'location' ]='Diego'
      data2.loc[data2['clean_text'].str.contains('(?i)cotland')==True,'location' ]='Scotland'
      data2.loc[data2['clean_text'].str.contains('(?i)rancisco')==True,'location' ]='Francisco'
      data2.loc[data2['clean_text'].str.contains('(?i)eorgia')==True,'location' ]='Georgia'
      data2.loc[data2['clean_text'].str.contains('(?i)allas')==True,'location' ]='Dallas'
      data2.loc[data2['clean_text'].str.contains('(?i)ancouver')==True,'location' ]='Dancouver'
      data2.loc[data2['clean_text'].str.contains('(?i)elbourne')==True,'location' ]='Melbourne'
      data2.loc[data2['clean_text'].str.contains('(?i)ingdom')==True,'location' ]='Kingdom'
      data2.loc[data2['clean_text'].str.contains('(?i)algary')==True,'location' ]='Calgary'
      data2.loc[data2['clean_text'].str.contains('(?i)apan')==True,'location' ]='Japan'
      data2.loc[data2['clean_text'].str.contains('(?i)hilippines')==True,'location' ]='Philippines'
      data2.loc[data2['clean_text'].str.contains('(?i)klahoma')==True,'location' ]='Oklahoma'
      data2.loc[data2['clean_text'].str.contains('(?i)ortland')==True,'location' ]='Portland'
      data2.loc[data2['clean_text'].str.contains('(?i)acramento')==True,'location' ]='Sacramento'
      data2.loc[data2['clean_text'].str.contains('(?i)oston')==True,'location' ]='Hoston'
      data2.loc[data2['clean_text'].str.contains('(?i)ustin')==True,'location' ]='justin'
      data2.loc[data2['clean_text'].str.contains('(?i)ashville')==True,'location' ]='ashville'
      data2.loc[data2['clean_text'].str.contains('(?i)hicago')==True,'location' ]='chicago'
      data2.loc[data2['clean_text'].str.contains('(?i)merica')==True,'location' ]='America'
      data2.loc[data2['clean_text'].str.contains('(?i)Vegas')==True,'location' ]='Vegas'
      data2.loc[data2['clean_text'].str.contains('(?i)emphis')==True,'location' ]='Memphis'
      data2.loc[data2['clean_text'].str.contains('(?i)Ohio')==True,'location' ]='Ohio'
      data2.loc[data2['clean_text'].str.contains('(?i)ictoria')==True,'location' ]='Victoria'
      data2.loc[data2['clean_text'].str.contains('(?i)ONTARIO')==True,'location' ]='Ontario'
      data2.loc[data2['clean_text'].str.contains('(?i)reland')==True,'location' ]='Ireland'
      data2.loc[data2['clean_text'].str.contains('(?i)irmingham')==True,'location' ]='Birmingham'
      data2.loc[data2['clean_text'].str.contains('(?i)ennessee')==True,'location' ]='Tennessee'
      data2.loc[data2['clean_text'].str.contains('(?i)ewcastle')==True,'location' ]='Newcastle'
      data2.loc[data2['clean_text'].str.contains('(?i)olumbia')==True,'location' ]='Columbia'
      data2.loc[data2['clean_text'].str.contains('(?i)Spain')==True,'location' ]='Spain'
      data2.loc[data2['clean_text'].str.contains('(?i)ansas')==True,'location' ]='Kansas'
      data2.loc[data2['clean_text'].str.contains('(?i)Denver')==True,'location' ]='Colorado'
      
      data2.loc[data2['clean_text'].str.contains('(?i)Louis')==True,'location' ]='Louis'

      data2.loc[data2['clean_text'].str.contains('(?i)rance')==True,'location' ]='France'
      data2.loc[data2['clean_text'].str.contains('(?i)etherlands')==True,'location' ]='Netherlands'
      data2.loc[data2['clean_text'].str.contains('(?i)weden')==True,'location' ]='Sweden'
      data2.loc[data2['clean_text'].str.contains('(?i)akistan')==True,'location' ]='Pakistan'
      data2.loc[data2['clean_text'].str.contains('(?i)ritish')==True,'location' ]='British'
      data2.loc[data2['clean_text'].str.contains('(?i)brasil')==True,'location' ]='Brasil'
      data2.loc[data2['clean_text'].str.contains('(?i)rome')==True,'location' ]='Rome'
      data2.loc[data2['clean_text'].str.contains('(?i)leveland')==True,'location' ]='cleveland'
      data2.loc[data2['clean_text'].str.contains('(?i)valley')==True,'location' ]='alley'
      data2.loc[data2['clean_text'].str.contains('(?i)ingapore')==True,'location' ]='singapore'
      data2.loc[data2['clean_text'].str.contains('(?i)arizona')==True,'location' ]='arizona'
      data2.loc[data2['clean_text'].str.contains('(?i)llinois')==True,'location' ]='illinois'
      data2.loc[data2['clean_text'].str.contains('(?i)tokyo')==True,'location' ]='tokyo'

In [ ]:
#agarramos  las localizaciones con null para completarlas(todas ellas tiene tweet)

In [60]:
data_only_null_location=data[data['location'].isna()]

In [67]:
test_only_null_location=test[test['location'].isna()]

In [68]:
test_only_null_location.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1105 entries, 0 to 3262
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1105 non-null   int64 
 1   keyword     1079 non-null   object
 2   location    0 non-null      object
 3   text        1105 non-null   object
 4   clean_text  1105 non-null   object
dtypes: int64(1), object(4)
memory usage: 51.8+ KB


In [61]:
data_only_null_location.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2533 entries, 0 to 7612
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2533 non-null   int64 
 1   keyword     2472 non-null   object
 2   location    0 non-null      object
 3   text        2533 non-null   object
 4   target      2533 non-null   int64 
 5   clean_text  2533 non-null   object
dtypes: int64(2), object(4)
memory usage: 138.5+ KB


In [62]:
data_only_null_location.head(5)

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [63]:
#hallamos las localizaciones dentro del tweet para completar los null de locationes
unificar_localization_with_tweet(data_only_null_location)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [69]:
#hallamos las localizaciones dentro del tweet para completar los null de locationes
unificar_localization_with_tweet(test_only_null_location)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [70]:
test_only_null_location['location'].value_counts()

New York, NY     52
California       17
America          15
Nigeria          11
Japan             9
India             7
France            6
Calgary           6
Massachusetts     6
Pakistan          6
Washington        4
Louis             4
Georgia           4
chicago           3
Australia         3
justin            3
Canada            2
Florida           2
Colorado          2
Oklahoma          2
England           1
alley             1
Oregon            1
British           1
Ireland           1
Birmingham        1
ashville          1
Portland          1
Newcastle         1
Rome              1
singapore         1
Columbia          1
tokyo             1
Texas             1
beattle           1
Name: location, dtype: int64

In [64]:
data_only_null_location['location'].value_counts()

New York, NY     105
California        50
Japan             30
India             24
Pakistan          20
France            19
Massachusetts     16
America           16
Florida           12
Nigeria           11
justin             9
Australia          9
Calgary            8
Canada             7
Texas              6
Georgia            6
chicago            6
Africa             6
colorado           5
Louis              5
Washington         4
Oklahoma           4
British            3
Rome               3
Hoston             3
toronto            3
Diego              2
Scotland           2
ashville           2
Sacramento         2
cleveland          2
Tennessee          2
Ontario            2
carolina           2
Memphis            2
Sweden             2
alley              2
Dallas             1
tokyo              1
sydney             1
Houston            1
England            1
Oregon             1
Michigan           1
Kingdom            1
Ireland            1
Victoria           1
Arizona      

In [65]:
#vemos que completo muchas location con el tweet que hicieron, tendra falsos positivos, pero pocos.
#ahora vamos a mergear los datos sin null qeu sacamos antes con este nuevo data set cque completo mas nuellos de location
datasinnullosenlocation=data[data['location'].isna()==False]
#rehacemos el dataset denuevo mas completo
location_complete_with_tweet=datasinnullosenlocation.append(data_only_null_location,ignore_index=True)

In [71]:
#vemos que completo muchas location con el tweet que hicieron, tendra falsos positivos, pero pocos.
#ahora vamos a mergear los datos sin null qeu sacamos antes con este nuevo data set cque completo mas nuellos de location
datasinnullosenlocation_test=test[test['location'].isna()==False]
#rehacemos el dataset denuevo mas completo
location_complete_with_tweet_test=datasinnullosenlocation_test.append(test_only_null_location,ignore_index=True)

In [72]:
location_complete_with_tweet_test.groupby('id').agg(canti=('id','count')).nlargest(5,'canti')

,canti
id,
0,1
2,1
3,1
9,1
11,1


In [66]:
#con esto confirmamos lo evidente, que no hay id duplicadas
location_complete_with_tweet.groupby('id').agg(canti=('id','count')).nlargest(5,'canti')

,canti
id,
1,1
4,1
5,1
6,1
7,1


In [73]:
#guardo este dataset , acordarse que este puede tener unos cuantos falsos positivos
location_complete_with_tweet_test.to_csv('test_uni_loca_2.csv') 
files.download('test_uni_loca_2.csv')
location_complete_with_tweet.to_csv('data_uni_loca_2.csv') 
files.download('data_uni_loca_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#ahora la idea es aproximar las localidades nulas que quedaron, mediante su tweet,
# es decir vamos a buscar el tweet mas similar con location completa,
#vamos a utilzar su location para completar el null de la otra location.
#para eso usamos embedding word2vec y la distancia entre vectores, la minima

In [74]:
!pip install gensim
from gensim.models import KeyedVectors

In [75]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-08-02 18:01:02--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.82.134
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.82.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  86.4MB/s    in 23s     

2020-08-02 18:01:25 (69.0 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [76]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#si queremos hacer el test, poner  data=location_complete_with_tweet_test

In [ ]:
#el pongo este nombre porque asi lo utilice para probar esta funcionablidad, pero  el 
#dataset ya esta completo con localizaciones unificadas y locationes que estaba nullos 
#completadas con valores de su tweet de locationes.
data=location_complete_with_tweet

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3263 non-null   int64 
 1   keyword     3237 non-null   object
 2   location    2337 non-null   object
 3   text        3263 non-null   object
 4   clean_text  1105 non-null   object
dtypes: int64(1), object(4)
memory usage: 127.6+ KB


In [ ]:
#vemos que hay 2533 nullos en location, puede ser peligroso aproximar esa cantidad
#asi que seguramente aproxime unos 500 localizaciones
data[data['location'].isna()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 926 entries, 2158 to 3261
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          926 non-null    int64 
 1   keyword     904 non-null    object
 2   location    0 non-null      object
 3   text        926 non-null    object
 4   clean_text  926 non-null    object
dtypes: int64(1), object(4)
memory usage: 43.4+ KB


In [ ]:
data[data['location'].notnull()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2337 entries, 0 to 3262
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2337 non-null   int64 
 1   keyword     2333 non-null   object
 2   location    2337 non-null   object
 3   text        2337 non-null   object
 4   clean_text  179 non-null    object
dtypes: int64(1), object(4)
memory usage: 109.5+ KB


In [80]:
def completar_location_by_similiaridad_tweet(data):
    sacar_columnas_usuales_final=data.groupby('location').agg(cantidad=('location','count')).nlargest(30,'cantidad')[:50]
    sacar_columnas_usuales_final=list(sacar_columnas_usuales_final.index)
    #esas columnas mas usuales son para que filtre el dataset, y entonces el ciclo sesa mas corto
    data_sin_null_location=data[data['location'].isin(sacar_columnas_usuales_final)==True]
    data_sin_null_location=data_sin_null_location[data_sin_null_location['location'].notnull()==True]
    #vamos aproximar una cantidad de 10% de valores nullos de location(esto podriamos variar,a mas o menos porcentaje si queremos)
    #total_de_nullos_a_completar=total_nullos*10/100  

    #total_de_nullos_a_completar=total_nullos*10/100
    total_de_nullos_a_completar=data[data['location'].isna()==True]['id'].count()*5/100
    total_de_nullos_a_completar=int(total_de_nullos_a_completar)

    #vamos agarrar solo 200 para predecir y completar
    data_with_null_location=data[data['location'].isna()==True][:total_de_nullos_a_completar]

    #en estos for comparo similaridad entre documento mediante word2vec, esto lo hace por distancia
    #vectorial

    acumlisttweetsimilar=list()
    acumidlisttweetsimilar=list()


    for i in range(len( data_with_null_location['text'])):
      distanciaminima=10000
    
      
      for j in range(len( data_sin_null_location['text'])):
        
        distancia_entre_vectores=word2vec.wmdistance(data_sin_null_location['text'].iloc[j], data_with_null_location['text'].iloc[i])
        
        if(distanciaminima>distancia_entre_vectores):
          distanciaminima=distancia_entre_vectores
        
          indexsinnull=j
          indexconnull=i
    
        if(j==(len( data_sin_null_location['text'])-1)) :
          idconnull=data_with_null_location['id'].iloc[indexconnull]
          idsinnull=data_sin_null_location['id'].iloc[indexsinnull]
          acumlisttweetsimilar.append( (data_sin_null_location['text'].iloc[indexsinnull], data_with_null_location['text'].iloc[indexconnull]))
          acumidlisttweetsimilar.append((data_sin_null_location['id'].iloc[indexsinnull],data_with_null_location['id'].iloc[indexconnull]))
          data_with_null_location.loc[data_with_null_location['id']==idconnull , 'location'] =list(data_sin_null_location.loc[data_sin_null_location['id']==idsinnull , 'location'])
          print(j)

    #aca vemos cuales fueron los tweet similares,descomentar para verlo
    #acumlisttweetsimilar[:5] 

    #aca las id de los tweet similares,descomentar para verlo
    #acumidlisttweetsimilar#(#sin, #con)     

    data_with_null_location_quedaron_sin_rellenar=data[data['location'].isna()==True][total_de_nullos_a_completar:]
    data_loca_relleandos=data_with_null_location
    data_null_and_fill_location=data_loca_relleandos.append(data_with_null_location_quedaron_sin_rellenar)
    #agarramos el dataset original con las unificaciones del paso 1 y 2, para despues agregar
    #el dataframe anterior
    data_sin_nullos_original=data[data['location'].isna()==False]

    data_parte3=data_sin_nullos_original.append(data_null_and_fill_location)
    return  data_parte3   

In [81]:
#vamos a agregar locaciones a el train
data=location_complete_with_tweet

In [83]:
train_parte_3=completar_location_by_similiaridad_tweet(data)

2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189
2189


In [85]:
train_parte_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7611
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          7613 non-null   int64 
 1   keyword     7552 non-null   object
 2   location    5610 non-null   object
 3   text        7613 non-null   object
 4   target      7613 non-null   int64 
 5   clean_text  7613 non-null   object
dtypes: int64(2), object(4)
memory usage: 416.3+ KB


In [87]:
location_complete_with_tweet_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3263 non-null   int64 
 1   keyword     3237 non-null   object
 2   location    2337 non-null   object
 3   text        3263 non-null   object
 4   clean_text  3263 non-null   object
dtypes: int64(1), object(4)
memory usage: 127.6+ KB


In [88]:
data=location_complete_with_tweet_test

In [89]:
test_parte_3=completar_location_by_similiaridad_tweet(data)

930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930
930


In [90]:
#csv de train y test con locaciones completadas con tweet similares a ellos.
from google.colab import files
train_parte_3.to_csv('train_uni_loca_3.csv') 
files.download('train_uni_loca_3.csv')

test_parte_3.to_csv('test_uni_loca_3.csv') 
files.download('test_uni_loca_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>